In [ ]:
from energy.maximum_energy_calculate import maximum_energy_calculate
import pandas as pd
plc = [40]
ts = maximum_energy_calculate(plc)
print(ts)
if isinstance(ts, pd.DataFrame):
    ts.to_csv('try.csv')

In [1]:
from clock.get_current_time import get_current_time
from clock.time_count import time_count
stime = '2022-09-04 '+get_current_time()[1]
end_time = get_current_time()[0]
time_count(stime, end_time)

0

In [5]:
from energy.maximum_energy_calculate import maximum_energy_calculate
from energy.median_energy_calculate import median_energy_calculate
from clock.get_current_time import get_current_time
import datetime
from settings import fmt1
start_time = '2022-09-16 14:00:00'
end_time = get_current_time()[0]
median_energy_calculate(start_time, end_time, [40])

***connected!***
***data queried!***
                 time  current  forward  reverse  forward.diff  reverse.diff
0 2022-09-17 00:01:42    -8.16  81666.0  69376.0           NaN           NaN
1 2022-09-17 00:30:42    -7.96  81694.0  69376.0          28.0           0.0
                 time  current  forward  reverse  forward.diff  reverse.diff
0 2022-09-17 00:01:42    -8.16  81666.0  69376.0           NaN           NaN
1 2022-09-17 00:30:42    -7.96  81694.0  69376.0          28.0           0.0


[28.0, 0.0]

In [2]:
from mongodb.query_request import query_request
import datetime
from settings import fmt1, filter_for_energy_calculation
start_time = '2022-09-15 08:00:00'
end_time = '2022-09-15 08:05:00'
start_time_ts = datetime.datetime.strptime(start_time, fmt1)
end_time_ts = datetime.datetime.strptime(end_time, fmt1)
data = query_request.datalog_query(start_time_ts, end_time_ts, [48], filter_for_energy_calculation)
data

***connected!***
***data queried!***


[{'PLC_id': 48,
  'time': datetime.datetime(2022, 9, 15, 8, 0, 20),
  'data': {'PCS电池电流': 0,
   '正向有功总电能': 33,
   '正向有功总电能2': 43665,
   '反向总有功电能': 29,
   '反向总有功电能2': 32985}},
 {'PLC_id': 48,
  'time': datetime.datetime(2022, 9, 15, 8, 1, 20),
  'data': {'PCS电池电流': 0,
   '正向有功总电能': 33,
   '正向有功总电能2': 43665,
   '反向总有功电能': 29,
   '反向总有功电能2': 32985}},
 {'PLC_id': 48,
  'time': datetime.datetime(2022, 9, 15, 8, 2, 20),
  'data': {'PCS电池电流': 7436,
   '正向有功总电能': 33,
   '正向有功总电能2': 43665,
   '反向总有功电能': 33,
   '反向总有功电能2': 43665}},
 {'PLC_id': 48,
  'time': datetime.datetime(2022, 9, 15, 8, 3, 20),
  'data': {'PCS电池电流': 7441,
   '正向有功总电能': 33,
   '正向有功总电能2': 43665,
   '反向总有功电能': 29,
   '反向总有功电能2': 33075}},
 {'PLC_id': 48,
  'time': datetime.datetime(2022, 9, 15, 8, 4, 20),
  'data': {'PCS电池电流': 7416,
   '正向有功总电能': 33,
   '正向有功总电能2': 43665,
   '反向总有功电能': 29,
   '反向总有功电能2': 33195}}]

In [ ]:
from sched import scheduler
from clock.get_current_time import get_current_time
from energy.energy_trigger import energy_trigger
from apscheduler.schedulers.blocking import BlockingScheduler
start_time = get_current_time()[0]
def et_try():
    print('---Begin---')
    plc = [2]
    flag = energy_trigger(plc)
    print(get_current_time()[0], flag)
    print('---end---')
scheduler = BlockingScheduler()
scheduler.add_job(et_try, 'interval', seconds = 2)
scheduler.start()

In [ ]:
from sched import scheduler
from clock.get_current_time import get_current_time
from energy.energy_trigger import energy_trigger
from apscheduler.schedulers.blocking import BlockingScheduler
from plcid.query_all_plc import query_all_plc
from settings import plc_csv_file, plc_csv_path
import pandas as pd
start_time = get_current_time()[0]
def diagnose_trigger_service():
    print('---Begin---')
    plc_update = query_all_plc()
    df_plc = pd.read_csv(plc_csv_path+plc_csv_file)
    plc_set = df_plc['plc'].values
    for i in range(0, df_plc.shape[0]):
        plc = [int(plc_set[i])]
        print(plc)
        flag = energy_trigger(plc)
        print(get_current_time()[0], flag)
    print('---end---')
scheduler = BlockingScheduler()
scheduler.add_job(diagnose_trigger_service, 'interval', seconds = 15)
scheduler.start()

In [14]:
import pandas as pd
a = pd.DataFrame({'A':[1,2,3], 'B':[1,2,3]})
b = pd.DataFrame({'A':[4,5,6], 'B':[4,5,6]})
c = pd.concat([a,b], axis=0, ignore_index=True)

In [ ]:
from messager.dingtalk_chatbot import dingtalkchatrobot
import pandas as pd
from settings import plc_csv_file, plc_csv_path
#dingtalkchatrobot.energy_trigger_sender()
df_plc = pd.read_csv(plc_csv_path+plc_csv_file)
plcset = df_plc['plc'].values
minus_set = []
zero_set = []
one_set = []
for plc in plcset:
    plc = [plc]
    energy_flag = dingtalkchatrobot.get_trigger(plc)
    if energy_flag == 0:
        zero_set.append(energy_flag)
    if energy_flag == -1:
        zero_set.append(energy_flag)
    if energy_flag == 1:
        zero_set.append(energy_flag)
print('0:', zero_set)
print('-1:', minus_set)
print('1:', one_set)

In [1]:
from mongodb.query_request import query_request
from clock.get_current_time import get_current_time
import datetime
from settings import energy_trigger_sender_webhook, \
    energy_trigger_sender_secret, bot_query_dayrange, filter_for_energy_trigger, fmt1
now_time = get_current_time()[0]
now_time_ts = datetime.datetime.strptime(now_time, fmt1)
start_time_ts = now_time_ts - datetime.timedelta(days=bot_query_dayrange)
print(start_time_ts ,now_time_ts)
data = query_request.trigger_query(start_time_ts, now_time_ts, [2], \
    filter_for_energy_trigger)

2022-09-13 11:28:56 2022-09-14 11:28:56
***connected!***
***trigger queried!***


In [6]:
threshold = [1,1]
max = [2,2]
plc = [2]
text_report = 'PLC_id:'+str(plc[0])+', '+'[充电量标准]forward:'+str(threshold[0])+', [放电量标准]reverse:'+str(threshold[0])+',\n'\
                +'[24小时内最大充电量]forward:'+str(max[0])+', [24小时内最大放电量]reverse:'+str(max[1])+'.'
print(text_report)

PLC_id:2, [充电量标准]forward:1, [放电量标准]reverse:1,
[24小时内最大充电量]forward:2, [24小时内最大放电量]reverse:2.


In [2]:
import pandas as pd
df = pd.DataFrame({'A':[-1,-2,-3]})
b = abs(df['A']).max()
c = df[abs(df['A'])>1].index
col = df.columns.get_loc('A')
df.iloc[c.values, col] = 0
df

,A
0,-1
1,0
2,0


In [18]:
from mongodb.query_request import query_request
from settings import plc_csv_file, plc_csv_path, filter_for_energy_threshold, fmt1
import pandas as pd
from clock.get_current_time import get_current_time
import datetime
df_plc_list = pd.read_csv(plc_csv_path+plc_csv_file)
plcset = df_plc_list['plc'].values
df_thres = pd.DataFrame()
for i in range(0, len(plcset)):
    plc = [int(plcset[i])]
    print(plc)
    start_time = str(df_plc_list[df_plc_list['plc']==plc[0]]['time'].values[0])
    start_time_ts = datetime.datetime.strptime(start_time, fmt1)
    now_time = get_current_time()
    now_time_ts = datetime.datetime.strptime(now_time[0], fmt1)
    data = query_request.threshold_query(start_time_ts, now_time_ts, plc, filter_for_energy_threshold)
    if len(data) > 0:
        thres_f = data[0]['median_max_forward']
        thres_r = data[0]['median_max_reverse']
        if thres_f >= 0 and thres_r >= 0:
            print(start_time_ts)
            thres_end_time_ts = start_time_ts + datetime.timedelta(days=7)
            print(thres_end_time_ts)
            to_append = pd.DataFrame({'plc': [plc[0]], '开始时间': [start_time_ts], '结束时间': [thres_end_time_ts], \
                                      '最大单次充电量': [thres_f], '最大单次放电量': [thres_r]})
            df_thres = df_thres.append(to_append)
df_thres = df_thres.sort_values(by = 'plc', ascending=True)
df_thres = df_thres.reset_index(drop=True)
df_thres = df_thres.loc[:, ~df_thres.columns.str.contains("^Unnamed")]
df_thres.to_csv('有效最大单次充放电量.csv')

[2]
***connected!***
***threshold queried!***
[4]
***connected!***
***threshold queried!***
[5]
***connected!***
***threshold queried!***
[7]
***connected!***
***threshold queried!***
[9]
***connected!***
***threshold queried!***
2022-09-06 12:42:20
2022-09-13 12:42:20
[17]
***connected!***
***threshold queried!***
[19]
***connected!***
***threshold queried!***
2022-09-06 12:42:20
2022-09-13 12:42:20
[18]
***connected!***
***threshold queried!***
2022-09-06 12:42:20
2022-09-13 12:42:20
[8]
***connected!***
***threshold queried!***
[14]
***connected!***
***threshold queried!***
[21]
***connected!***
***threshold queried!***
2022-09-06 12:42:20
2022-09-13 12:42:20
[22]
***connected!***
***threshold queried!***
[24]
***connected!***
***threshold queried!***
[25]
***connected!***
***threshold queried!***
[28]
***connected!***
***threshold queried!***
[33]
***connected!***
***threshold queried!***
2022-09-06 12:42:20
2022-09-13 12:42:20
[32]
***connected!***
***threshold queried!***
2022-09

In [1]:
from data.dron import dron
data = dron.find_battery_to_check([44])

***connected!***
***connected!***
***data queried!***


In [2]:
data['Voltage'].mean()

3.399291666666667

In [7]:
data['Voltage'].std()

0.003906284420138223

In [4]:
data

,time,Voltage.name,Voltage,Current,Temperature,dv
693604,2022-09-26 11:30:30,10组7号电池电压,3.399,-69.04,31.0,0.000000
693627,2022-09-26 11:30:30,10组12号电池电压,3.401,-69.04,31.0,0.511995
693629,2022-09-26 11:30:30,10组19号电池电压,3.395,-69.04,31.0,-1.023991
693631,2022-09-26 11:30:30,10组4号电池电压,3.398,-69.04,31.0,-0.255998
693656,2022-09-26 11:30:30,10组11号电池电压,3.399,-69.04,31.0,0.000000
693663,2022-09-26 11:30:30,10组22号电池电压,3.399,-69.04,31.0,0.000000
693682,2022-09-26 11:30:30,10组9号电池电压,3.400,-69.04,31.0,0.255998
693688,2022-09-26 11:30:30,10组21号电池电压,3.398,-69.04,31.0,-0.255998
693696,2022-09-26 11:30:30,10组15号电池电压,3.401,-69.04,31.0,0.511995
693709,2022-09-26 11:30:30,10组23号电池电压,3.404,-69.04,31.0,1.279989


In [10]:
(3.410-data['Voltage'].mean())/data['Voltage'].std()

2.741309178135622

In [11]:
2.815975*data['Voltage'].std()+data['Voltage'].mean()

3.4102916659366658